# Исследование игровой индустрии 

В нашем распоряжении исторические данные из открытых источников о продажах игр, оценки пользователей и критиков, жанрах и платформ. Целью данного исследования является изучение закономерностей, которые определяют успешность определенной игры. Задача исследования - проверить гипотезы, позволяющие определить равны ли пользовательские рейтинги платформ Xbox & PC, жанров Action & Sports.

План работы включает в себя:
- предобработку данных: определение пустых строк, дублирущих значений; 
- исследовательский анализ данных с помощью построения различных видов графиков;
- составление портрета пользователя на основе данных регионов: Северная Америка, Европа, Япония и других стран.
- проверку статистических гипотез.

Данные хранятся в файле games.csv, представлена информация за период с 1980 года по 2016 год - мы условно находимся в декабре 2016 года. Нам необходимо исследовать датафрейм для того, чтобы спрогнозировать продажи в 2017 году.

Описание колонок:
- Name — название игры;
- Platform — платформа;
- Year_of_Release — год выпуска;
- Genre — жанр игры;
- NA_sales — продажи в Северной Америке (миллионы проданных копий);
- EU_sales — продажи в Европе (миллионы проданных копий);
- JP_sales — продажи в Японии (миллионы проданных копий);
- Other_sales — продажи в других странах (миллионы проданных копий);
- Critic_Score — оценка критиков (максимум 100);
- User_Score — оценка пользователей (максимум 10);
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Обзор данных

#### Импорт библиотек, выведение первых строк таблиц и изучение полученной информации

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline
init_notebook_mode(connected=True)

import scipy.stats as st

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'seaborn'

In [2]:
def my_func(x):
    print('Исходный датафрейм')
    display(x.head())
    print('Общая информация о датафрейме')
    display(x.info())
    print('Количество пустых значений в датафрейме')
    display(x.isna().sum())
    print('Количество явных дубликатов в датафрейме')
    display(x.duplicated().sum())
    print('Описательная статистика')
    display(x.describe())
    

try: 
    data = pd.read_csv('/datasets/games.csv')
except:
    print('Не удалось открыть датасет')
my_func(data)

Не удалось открыть датасет


NameError: name 'data' is not defined

Для удобства создадим переменную, которая содержит в себе информацию о столбцах, в которых присутствуют пустые значения.

In [ ]:
all_null = data.isna().sum().to_frame(name='nans').query('nans > 0')['nans']
all_null

**Вывод:**

При первичном анализе было выявлено, что в нескольких столбцах есть пустые значения, в некоторых из которых их достаточное количество: название игры (2 строки), год выпуска (269), жанр (2), оценка критиков (8578), оценка пользователей (6701), рейтинг от ESRB (6766).

Явные аномалии не найдены, единственное, что можно отметить - что присутствуют нулевые продажи в Северной Америке, Европе, Японии и других странах, вероятнее всего из-за пропущенных значений, что проверим в ходе исследования.

В дальнейшем необходимо поменять тип данных в следующих столбцах:
- год выпуск: год переведем в формат числа, так как не может быть, например, "2023,4" год;
- оценка пользователей: переведем в вещественный тип;
- округлим значения продаж до одной цифры после запятой.

### Предобработка данных

#### Изменение названия столбцов

Выполнили форматирование столбцов в датафреме путем приведения названий столбцов к нижнему регистру.

In [ ]:
data.columns = [x.lower() for x in data.columns]
data.head()

#### Преобразование типов данных

Столбец с годом выпуска компьютерных игр преобразуем после обработки пустых значений. Столбец с оценкой пользователей переведем в вещественный тип. Для начала выведем уникальные значения в данном столбце, чтобы правильно выбрать метод форматирования.

In [ ]:
data['user_score'].unique()

Видим, что у нас есть не только пустые строки "NaN", но и строки с содержанием "tbd", их 2424:

In [ ]:
data.loc[data['user_score'] == 'tbd']

И по ним нельзя явно сказать, что это за значение, поэтому заменим на "NaN".

In [ ]:
data['user_score'] = pd.to_numeric(data['user_score'] , errors='coerce')

Также округлим значения продаж до одной цифры после запятой.

In [ ]:
to_round = {'na_sales', 'eu_sales', 'jp_sales', 'other_sales'}
for o in to_round:
        data[o] = round(data[o], 1)

#### Обработка пропущенных значений

Пропущенные значения хранятся в следующих столбцах:

In [ ]:
all_null

##### Столбец 'name'

Пойдем по порядку и начнем со столбца с названиями игр, данных совсем мало, всего две строки, выведем их для подробного рассмотрения:

In [ ]:
data.loc[data['name'].isna()].head()

Из полученной таблицы мы не можем точно понять, какая именно игра это была. Так как это количество совсем несущественное, принимаем решение удалить строки. Обращаем внимание, что именно по данным двум строкам отсутствуют значения в столбце 'genre', поэтому удалив строки мы обработаем сразу два столцба с пропущенными значениями.

In [ ]:
data = data.dropna(subset=['name'])
data['name'].isna().sum()

Обновим индексы:

In [ ]:
data.reset_index(drop=True, inplace=True)

##### Столбец 'year_of_release'

Переходим к году выпуска компьютерной игры, где пропущенные значения присутствуют в 269 строках.

In [ ]:
data.loc[data['year_of_release'].isna()].head()

Можно принять во внимание тот факт, что мы обладаем информацией о названии игры, если данное название встречается на других платформах и там присутствует информация по году релиза - сможем ее подтянуть вместо наших пропусков. Проверим, так ли это на примере "Madden NFL 2004".

In [ ]:
data.loc[data['name'] == 'Madden NFL 2004'].head()

Да, мы убедились в этом, теперь мы можем заполнить пропуски, сначала создадим сводную таблицу с данными о годе выпуска компьютерных игр:

In [ ]:
data_pivot_year = pd.pivot_table(
    data,
    index='name',
    values='year_of_release',
    aggfunc='min'
)
data_pivot_year.reset_index(inplace=True)
data_pivot_year.rename(columns = {'name': 'name'})
data_pivot_year.head()

Теперь подтянем информацию по году релиза определенных игр в пустые значения, создав дополнительный столбец:

In [ ]:
data = data.merge(data_pivot_year, on='name', how='left', suffixes=('_before', '_after'))
data.head()

Проверим, есть ли пропуски в новом столбце:

In [ ]:
data['year_of_release_after'].isna().sum()

Все равно есть 146 пропущенных значений, проверим, что это за строки:

In [ ]:
data_146 = data.loc[data['year_of_release_after'].isna()]
data_146.head(10)

Вернемся к исходному датафрейму и попытаемся понять, причину пропусков на примере игры 'Rock Band'.

In [ ]:
data.loc[data['name'] == 'Rock Band'].head()

Видим, что компьютерная игра 'Rock Band' представлена на нескольких платформах, и мы не обладаем информацией по году релиза - оставим такие пропуски без изменений. Теперь оставим нужный столбец:

In [ ]:
data.drop('year_of_release_before', axis=1, inplace=True)
data = data.rename(columns = {'year_of_release_after':'year_of_release'})
data = data[['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', \
              'critic_score', 'user_score', 'rating']]
data.head()

Посчитаем, какое процентное соотношение пропусков в названиях к общему числу:

In [ ]:
m = data['year_of_release'].isna().sum()
n = data['year_of_release'].shape[0]
round((m/n)*100,2)

0.87% - очень низкий процент пропущенных строк, они никак не повлияют на наше исследование - поэтому принимаем решение удалить пропуски.

In [ ]:
data = data.dropna(subset=['year_of_release'])
data['year_of_release'].isna().sum()

Обновим индексы:

In [ ]:
data.reset_index(drop=True, inplace=True)

##### Столбец 'genre'

Столбец 'genre' мы обработали, обновим все данные по пропускам:

In [ ]:
data.isna().sum()

##### Столбцы 'critic_score', 'user_score' и 'rating'

В столбце 'critic_score' 8576 пропущенных значений - это самое большое количество по сравнению с другими столбцами. Рассмотрим его подробнее:

In [ ]:
data.query('critic_score.isnull()')

Проверим, сколько у нас пересечений по пропущенным данным в столбцах 'critic_score', 'user_score' и 'rating'.

In [ ]:
len(data.loc[(data['critic_score'].isna()) & (data['user_score'].isna()) & (data['rating'].isna())])

6619 строк - большое количество, возможно, эти данные с пропусками возникли по причине того, что рейтинг просто не стоит у той или иной игры, например, из-за того, что не было сервисов по оценке.

In [ ]:
m = len(data.loc[(data['critic_score'].isna()) & (data['user_score'].isna()) & (data['rating'].isna())])
n = data['critic_score'].shape[0]
round((m/n)*100,2)

И так как они составляют достаточно большой процент (почти 40%) от общего числа строк - удалять строки не будем, но этом мы не можем заменить их на медианное значение в разрезе жанров, что будет некорректно, поэтому заменим "NaN" на "-1", в рейтинге на "no rating", чтобы мы могли в дальнейшем отличать данные строки от остальных показателей рейтингов.

In [ ]:
to_convert = {'critic_score', 'user_score'}
for i in to_convert:
        data[i] = data[i].fillna(-1)
        
data['rating'] = data['rating'].fillna('no rating')

In [ ]:
data.head()

#### Обработка явных и неявных дубликатов

В начале исследования выявили, что явных дубликтов в датафрейме нет, что касается неявных, рассмотрим, есть ли в датафрейме одинаковые строки по названию игры, платформе и году выпуска. Предварительно запомним, какое количество строк в датафрейме:

In [ ]:
data.shape[0]

In [ ]:
data[data[['name', 'platform', 'year_of_release']].duplicated(keep=False)]

Строки по 4м играм задублированы - в них представлена одинаковая информация по названию игры, платформе, году релиза, жанру, оценкам пользователей и критиков, рейтингу, но при этом данные по продажам в разных регионах отличается. В данном случае нам необходимо сохранить данные по продажам, поэтому создадим сводную таблицу, просуммировав значения в соответствующих столбцах. Затем перезапишем датафрейм на основе нового свода данных:

In [ ]:
add = pd.pivot_table(
    data,
    index=['name', 'platform', 'year_of_release', 'genre', 'critic_score', 'user_score', 'rating'],
    values=['na_sales', 'eu_sales', 'jp_sales', 'other_sales'],
    aggfunc='sum'
    )
add.reset_index(inplace=True)
data = add[['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', \
             'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating']]
data.head()

In [ ]:
data.shape[0]

Проверили новый датафрейм - на 4 строки меньше, все верно. Теперь повторно выполним поиск на неявные дубликаты:

In [ ]:
data[data[['name', 'platform', 'year_of_release']].duplicated(keep=False)]

Избавились от неявных дубликатов.

#### Подсчет суммарных продаж во всех регионах

Добавим новый столбец в датафрейм с суммой продаж по всем регионам:

In [ ]:
data['total_sales'] = data[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']]\
.sum(axis = 1, skipna = True)

In [ ]:
data.head()

**Вывод:**

На данном этапе были сделаны следующие шаги:
- преобразованы типы данных для корректных рассчетов в дальнейшем исследовании;
- обработаны пропущенные значения в столбцах с названием игры & жанра - удалены две строки из датафрейма;
- обработаны пропущенные значения в столбце год выпуска - заполнены данные в местах, где это было возможно, остальные пропуски оставлены без изменений и заменены на "0".
- обработаны пропущенные значения в столбцах оценка критиков & оценка пользователей & рейтинг - пропущенные значения заменены на "-1" и "no rating";
- обработаны неявные дубликаты;
- подсчитаны суммарные продажи во всех регионах.

### Исследовательский анализ данных

#### Подсчет количества выпущенных компьютерных игр по годам

Создадим сводную таблицу с количеством компьютерных игр по годам:

In [ ]:
data_pivot_quantity = pd.pivot_table(
    data,
    index='year_of_release',
    values='name',
    aggfunc='count'
)

data_pivot_quantity.reset_index(inplace=True)
data_pivot_quantity = data_pivot_quantity\
.rename(columns = {'year_of_release': 'year_of_release', 'name': 'games_quantity'})
data_pivot_quantity

По таблице выше можно сделать вывод, что размах представленных данных от 1980 года до 2016. Визуализируем наши данные за исключением пропущенных значений.

In [ ]:
data_pivot_quantity = data_pivot_quantity[data_pivot_quantity['year_of_release'] > 0]

fig = go.Figure()

fig.add_trace(go.Scatter(x=data_pivot_quantity.year_of_release, 
                         y=data_pivot_quantity.games_quantity, 
                         mode='lines+markers', 
                         name='Quantity'))

fig.update_layout(title='Dependence of the release of games on the year of release',
                  xaxis_title='Yeas',
                  yaxis_title='Number of computer games'
                 )
fig.show("svg")

По графику выше можно сделать вывод, что динамичное развитие игровой индустрии началось в 2000-х годах, в 2008-2009 гг приходится пик выпуска компьютерных игр - немного больше 1400 игр в каждый год. Затем пошел спад после 2011 года с 1156 штук до 661 штук в 2012 году. Все данные брать в рассмотрение не стоит.

#### Изменение продаж по платформам

Теперь изучим, как менялись продажи по платформам.

In [ ]:
data_pivot_sales = pd.pivot_table(
    data,
    index='platform',
    values='total_sales',
    aggfunc='sum'
)

data_pivot_sales.reset_index(inplace=True)
data_pivot_sales = data_pivot_sales.sort_values(by='total_sales', ascending=False).reset_index(drop=True)
data_pivot_sales

Оставим топ-10 платформ и визуализируем данные на графике:

In [ ]:
data_pivot_sales = data_pivot_sales[:10]

fig = px.bar(data_pivot_sales,
             x='platform',
             y='total_sales',
             title='Dependence of computer game sales on the platform', 
             text='total_sales', 
             color='platform'
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8)
fig.update_layout(xaxis_tickangle=-45)
fig.update_layout(xaxis_title='Platform',
                  yaxis_title='Total sales'
                 )              
              
fig.show("svg")

Диаграмма выше демонстрирует, что лидером по продажам игр является платформа PS2 - почти 1200 млн проданных копий, следом за ней идут: X360 (950 млн) и PS3 (920 млн). Для дальнейшего исследования возьмем топ-6 платформ по количеству проданных копий и построим распределение по годам.

In [ ]:
data_top6 = data.query('year_of_release > 0 and platform == ["PS4", "X360", "PS3", "Wii", "DS", "PS"]')

data_pivot_sales_2 = pd.pivot_table(
    data_top6,
    index=['year_of_release','platform'],
    values='total_sales',
    aggfunc='sum'
)

data_pivot_sales_2.reset_index(inplace=True)
fig = px.line(data_pivot_sales_2, x='year_of_release', y='total_sales', color='platform')
fig.show("svg")

По графику выше можно сделать вывод, что по следующим платформам их время популярности в среднем составляло:
- PS: 1994-2003, 9 лет.
- PS3: 2005-2015, 10 лет.
- DS: 2004-2011, 7 лет.
- Wii: 2005-2015, 10 лет.
- X360: 2005-2015, 10 лет.

В среднем в течение 9 лет происходит "смена" платформ по их популярности.

#### Ограничение датафрейма по актуальным данным

Рынок компьютерных игр весьма динамичен, судя по графикам, представленным выше. Для дальнейшего исследования принимаем решение использовать данные за последние 4 года, то есть, мы находимся к декабре 2016, значит, наш диапазон данных будет с 2013 года. Так, мы сможем учесть достаточно актуальные и успешные платформы, которые в 2017 году смогут выйти на пик по популярности. Среди них и те, которые уже сейчас находятся на пике популярности, и те, которые только начинают расти в популярности.

In [ ]:
data_new = data.query('year_of_release >= 2013')

#### Определение лидирующих платформ за период 2013-2016гг

Построим сводную таблицу с продажами для определения лидирующих платформ в новом временном диапазоне и визуализируем данные:

In [ ]:
data_pivot_new = pd.pivot_table(
    data_new,
    index=['year_of_release','platform'],
    values='total_sales',
    aggfunc='sum'
)
data_pivot_new.reset_index(inplace=True)
data_pivot_new = data_pivot_new.sort_values(by='total_sales', ascending=False).reset_index(drop=True)

fig = px.bar(data_pivot_new,
             x='year_of_release',
             y='total_sales',
             title='Dependence of computer game sales on the platform for the period 2013-2016', 
             text='total_sales', 
             color='platform',
             barmode='group'
)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8)
fig.update_layout(xaxis_tickangle=-45)
fig.update_layout(xaxis_title='Platform',
                  yaxis_title='Total sales'
                 )              
              
fig.show("svg")

Судя по столбчатой диаграмме выше, можно сказать, что в период с 2013 по 2016 гг лидером по продажам была платформа PS4, XOne также показывает хорошие результаты по популярности, но прибыль различается от лидера в два раза. Общая тенденция следующая: к 2016 году все платформы устаревают и прибыль от продажи копий сильно снижается. Необходимо определить потенциально прибыльные платформы, которые могут "выстрелить" в следующем году - те, игры на которых были выпущены непосредственно в 2016 году.

In [ ]:
data_pivot_new.loc[data_pivot_new['year_of_release'] == 2016].sort_values('total_sales', ascending=False)\
.reset_index(drop=True)

Игры выпускались в 2016 году на следующих платформах в порядке убывания:
1. PS4
2. XOne
3. 3DS
4. WiiU
5. PC
6. PSV
7. PS3
8. X360
9. Wii

Делаем вывод, что наиболее актуальными платформами являются PS4, XOne и 3DS.

#### Построение диаграммы размаха по глобальным продажам игр в разбивке по платформам

Построим "ящик с усами" по всем платформам:

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(
            y=data_new['total_sales'],
            x=data_new['platform'],
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor='lavender',
            marker_size=2,
            line_width=1
))

fig.update_layout(
    title='Boxplot of global game sales by platform',
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
        range=[0,5],
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

fig.update_layout(xaxis_title='Platform',
                  yaxis_title='Totsl sales'
                 )              

fig.show("svg")

По полученной диаграмме размаха можно сделать вывод, что за период с 2013 по 2016гг:

- Глядя на медианы, видно, что наибольшие продажи у X360, PS4, XOne, WiiU, Wii.
- Глядя на размеры ящиков, видно, что наибольший разброс в продажах игр - на платформах X360, XOne, Wii.
- Выбросов достаточно много.

#### Влияние отзывов пользователей и критиков на продажи внутри одной популярной платформы

##### PS4

Для рассмотрения возьмем лидера - платформу PS4, построим диаграмму рассеяния и посчитаем корреляцию между отзывами и продажами. Сначала ограничим датафрейм - оставим только ту оценку, которая без пропущенных значений.

In [ ]:
data_scores = data_new.query('critic_score > -1 and user_score > -1')
data_scores = data_scores.loc[:, ['platform', 'critic_score', 'user_score', 'total_sales']]

Выведем общие данные по корреляции:

In [ ]:
corr = round(data_scores.corr(), 2)
corr

Для более удобного чтения подсветим корреляции:

In [ ]:
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

На первый взгляд оценка критиков имеет большее влияние на общие продажи, чем оценка пользователей, но сама корреляция незначительная - 0.32%. В то время как оценка пользователей вообще не влияет на продажи - 0%.

Теперь рассмотрим лидера PS4 - как влияют на продажи игр, выпущенных на данной платформе, оценка критиков и оценка пользователей последовательно.

In [ ]:
def data_pivot_corr(pltf, aggr_index, vls, f2):
    platform = data_scores[data_scores['platform'] == pltf]
    data_pivot_corr = platform.pivot_table(
        index = [aggr_index], 
        values = vls,
        aggfunc = {f2})
    data_pivot_corr.reset_index(inplace=True)
    data_pivot_corr = data_pivot_corr.sort_values(by=f2, ascending=True)
    return data_pivot_corr

corr_data1 = data_pivot_corr('PS4', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data1,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PS4',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data1['critic_score'].corr(corr_data1['median']))

Корреляция в 0.48 говорит о наличии связи, однако очень незначительной. Выходит, что увеличение размера оценки игры сопровождается увеличением ее продажи.

In [ ]:
corr_data2 = data_pivot_corr('PS4', 'user_score', 'total_sales', 'median')
px.scatter(corr_data2,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PS4',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data2['user_score'].corr(corr_data2['median']))

Корреляция в 0.008 говорит об очень слабой связи между оценкой пользователей и продажами игр на платформе PS4. При увеличении оценки незначительно увеличиваются продажи.

##### XOne

Проведем аналогичный анализ по остальным платформам:

In [ ]:
corr_data3 = data_pivot_corr('XOne', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data3,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе XOne',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data3['critic_score'].corr(corr_data3['median']))

Корреляция в 0.54 - слабая связь между оценкой критиков и продажами платформы.

In [ ]:
corr_data4 = data_pivot_corr('XOne', 'user_score', 'total_sales', 'median')
px.scatter(corr_data4,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе XOne',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data4['user_score'].corr(corr_data4['median']))

Отрицательная корреляция -0.13 говорит о том, что связи практически нет, при уменьшении оценки пользователей, продажи на игры падают.

##### 3DS

In [ ]:
corr_data5 = data_pivot_corr('3DS', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data5,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе 3DS',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data5['critic_score'].corr(corr_data5['median']))

Корреляция в 0.40 - есть небольшая связь между оценками критиков и продажами платформы 3DS.

In [ ]:
corr_data6 = data_pivot_corr('3DS', 'user_score', 'total_sales', 'median')
px.scatter(corr_data6,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе 3DS',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data6['user_score'].corr(corr_data6['median']))

Примерно такое же влияние оценки пользователей на продажи платформы 3DS - корреляция в 0.39.

##### WiiU

In [ ]:
corr_data7 = data_pivot_corr('WiiU', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data7,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе WiiU',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data7['critic_score'].corr(corr_data7['median']))

Что касается продаж на платформе WiiU, то чем больше оценка критиков на те или иные игры, тем выше продажи - есть связь, но все равно незначительнся, 0.52.

In [ ]:
corr_data8 = data_pivot_corr('WiiU', 'user_score', 'total_sales', 'median')
px.scatter(corr_data8,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе WiiU',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data8['user_score'].corr(corr_data8['median']))

Примерно такое же влияние оказывают оценки пользователей, корреляция между ними и продажами, 0.45.

##### PC

In [ ]:
corr_data9 = data_pivot_corr('PC', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data9,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PC',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data9['critic_score'].corr(corr_data9['median']))

На продажи на платформе PC незначительно влияют оценки критиков - корреляция равна 0.52.

In [ ]:
corr_data10 = data_pivot_corr('PC', 'user_score', 'total_sales', 'median')
px.scatter(corr_data10,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PC',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data10['user_score'].corr(corr_data10['median']))

В то время как связи между оценкой пользователей и продажами практически нет - корреляция отрицательная, -0.15: с уменьшением оценки пользователей, незначительно снижаются продажи игр на платформе.

##### PSV

In [ ]:
corr_data11 = data_pivot_corr('PSV', 'critic_score', 'total_sales', 'median')
px.scatter(corr_data11,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PSV',
           labels=dict(index='Оценка критиков', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data11['critic_score'].corr(corr_data11['median']))

Небольшая связь между оценкой критиков и продажами на платформе PSV, 0.36.

In [ ]:
corr_data12 = data_pivot_corr('PSV', 'user_score', 'total_sales', 'median')
px.scatter(corr_data12,
           y='median', 
           width=800, 
           height=500, 
           title='Зависимость оценки критиков на продажи компьютерных игр на платформе PSV',
           labels=dict(index='Оценка пользователей', median='Продажи'))
fig.show("svg")

In [ ]:
print('Корреляция равна ', corr_data12['user_score'].corr(corr_data12['median']))

Немного, но влияют оценки пользователей - корреляция 0.31, при увеличении оценки увеличиваются и продажи.

#### Распределение игр по жанрам

Составим сводную таблицу с распределением по жанрам:

In [ ]:
genres = pd.pivot_table(
    data_new,
    index='genre',
    values='total_sales',
    aggfunc={'median', 'count'}
)
genres = genres.sort_values(by='median', ascending=False)
genres

Согласно полученным данным, можно выделить жанры с высокими продажами: Shooter, Platform и Sports, и с низкими продажами (все остальные): Adventure, Strategy, Simulation, Role-Playing, Racing, Puzzle, Misc, Fighting, Action. Также можно обратить внимание на то, что есть некоторые жанры, игры по которым были выпущены в большом количестве, но при этом продажи не были высокими. Самым прибыльным считается жанр Shooter, что неудивительно, так как помимо покупки самой игры чаще всего игроки делают внутриигровые покупки на необходимую "экипировку" и т.д.

**Вывод:**

1. Динамичное развитие игровой индустрии началось в 2000-х годах, в 2008-2009 гг приходится пик выпуска компьютерных игр - немного больше 1400 игр в каждый год.
2. В среднем в течение 6 лет происходит "смена" платформ по их популярности.
3. В период с 2013 по 2016 гг лидером по продажам была платформа PS4, XOne также показывает хорошие результаты по популярности, но прибыль различается от лидера в два раза. Общая тенденция следующая: к 2016 году все платформы устаревают и прибыль от продажи копий сильно снижается.
4. Диаграмма рессеяния показала, что наибольшие продажи у X360, PS4, XOne, WiiU, Wii, наибольший разброс в продажах игр - на платформах X360, XOne, Wii. Выбросов достаточно много.
5. Зависимость между отзывами критиков и продажами на платформах PS4, XOne, 3DS, WiiU, PC, PSV гораздо сильнее, чем корреляция между отзывами пользователей и продажами на платформах.
6. Можно выделить жанры с высокими продажами: Shooter, Platform и Sports, и с низкими продажами (все остальные): Adventure, Strategy, Simulation, Role-Playing, Racing, Puzzle, Misc, Fighting, Action. Также можно обратить внимание на то, что есть некоторые жанры, игры по которым были выпущены в большом количестве, но при этом продажи не были высокими. Самым прибыльным считается жанр Shooter, что неудивительно, так как помимо покупки самой игры чаще всего игроки делают внутриигровые покупки на необходимую "экипировку" и т.д.

### Составление портрета пользователя каждого региона

Ограничим датафрейм по регионам:

In [ ]:
na = data_new[data_new['na_sales'] != 0]
eu = data_new[data_new['eu_sales'] != 0]
jp = data_new[data_new['jp_sales'] != 0]
other = data_new[data_new['other_sales'] != 0]

#### Топ-5 самых популярных платформ

Создадим функцию для расчета сводной таблицы по топ-5 самых популярных платформ в каждом регионе:

In [ ]:
def data_pivot_func(df, aggr_index, vls, f1, f2):
    data_pivot_func = pd.pivot_table(
        df,
        index=aggr_index,
        values=vls,
        aggfunc={vls:[f1, f2]}
    )
    data_pivot_func['abc'] = 'abc'
    data_pivot_func['total'] = (
    data_pivot_func.groupby('abc')['sum'].transform('sum')
    )
    data_pivot_func['ratio'] = (data_pivot_func['sum'] / data_pivot_func['total'])\
    .apply(lambda x: round(x * 100, 2))
    data_pivot_func = data_pivot_func.sort_values(by='ratio', ascending=False)
    data_pivot_func = data_pivot_func[:5]
    data_pivot_func = data_pivot_func.drop(['abc'], axis=1)
    data_pivot_func.reset_index(inplace=True)
    return data_pivot_func

na_pivot_platform = data_pivot_func(na, 'platform', 'na_sales', 'count', 'sum')
na_pivot_platform

В Северной Америке самая популярная платформа по доле продаж компьютерных игр - PS4, 24.9%, следом за ней идет XOne, 21.65%, затем X360, 19.07%, PS3, 14.15%. На пятом месте 3DS с большим отрывом - 8.72%.

In [ ]:
eu_pivot_platform = data_pivot_func(eu, 'platform', 'eu_sales', 'count', 'sum')
eu_pivot_platform

В Европе самая популярная платформа по доле продаж компьютерных игр также - PS4, но доля уже составила 36.47% - платформа пользуется спросом больше в Европе. Следом за ней идет PS3 с большим отрывом и долей продаж в 17.12%. На третьем месте XOne, 13.35%, затем X360, 10.82%. На пятом месте 3DS - 7.78%, она также непопулярна в этом регионе.

In [ ]:
jp_pivot_platform = data_pivot_func(jp, 'platform', 'jp_sales', 'count', 'sum')
jp_pivot_platform

В Японии самой популярной платформой становится 3DS - 51.77%. PS3 (15.56%), PSV (11.36%), PS4 (11.03%) вне конкуренции по сравнению с долей продаж игр на 3DS, замыкает пятерку лидеров платформа WiiU с долей продаж 8.15%.

In [ ]:
other_pivot_platform = data_pivot_func(other, 'platform', 'other_sales', 'count', 'sum')
other_pivot_platform

В остальных странах самая популярная платформа по доле продаж компьютерных игр сохраняется - PS4, с высоким процентом 43.29%, следом за ней идет PS3 (24.06%) и XOne (11.21%). X360 (9.86%) и 3DS (4.25%) замыкают пятерку лидеров.

**Вывод:**

PS4 остается самой популярной платформой в Северной Америке и Европе и других странах. В то время как в Японии лидирует платформа 3DS. Доля продаж таких платформ, как XOne, PS3, X360, PSV средняя по всем регионам.

#### Топ-5 самых популярных жанров

Рассмотрим долю продаж самых популярных жанров:

In [ ]:
na_pivot_genre = data_pivot_func(na, 'genre', 'na_sales', 'count', 'sum')
na_pivot_genre

В Северной Америке доля продаж компьтерных игр в жанре Action самая большая - 28.94%, на втором месте Shooter с разницей всего в 3% - 25.95%. Жанр Sports не мене популярен, хоть и доля продаж составляет 15.65%. Замыкают пятерку лидеров игры в жанрах Role-Playing(9.15%) и Misc (6.50%).

In [ ]:
eu_pivot_genre = data_pivot_func(eu, 'genre', 'eu_sales', 'count', 'sum')
eu_pivot_genre

В Европе лидером по продажам также является игры в жанре Action (30.31%). Shooter (23.34%) на втором месте, Sports (15.88%) на третьем. Замыкают Role-Playing (8.16%) и Racing (5.22%).

In [ ]:
jp_pivot_genre = data_pivot_func(jp, 'genre', 'jp_sales', 'count', 'sum')
jp_pivot_genre

В Японии пользуются большим спросом игры в жанрах Role-Playing (35.56%) и Action (30.12%). С большим отрывом игры в жанрах Misc (7.16%), Fighting (5.35%) и Shooter (4.94%).

In [ ]:
other_pivot_genre = data_pivot_func(other, 'genre', 'other_sales', 'count', 'sum')
other_pivot_genre

Что касается других стран, то популярными в них являются игры в жанрах Action (31.30%) и Shooter (26.28%). С небольшим отрывом на третьем месте Sports (17.20%). Замыкают пятерку лидеров игры в жанрах Role-Playing и Misc с долями продаж 7.44% и 4.93% соответственно.

**Вывод:**

Компьютерные игры в жанрах Action и Shooter остаются на первых местах по доле продаж среди таких регионов, как Северная Америка, Европа и в других странах. В Японии игры в жанре Action остаются на втором месте по популярности, а на первое выходит жанр Role-Playing с долей продаж 35%. Жанры Sports, Role-Playing, Misc встречаются почти во всех регионах и находятся на примерно одном уровне популярности.

#### Влияние рейтинга ESRB на продажи в отдельном регионе

Теперь рассмотрим, как рейтинг ESRB влияет на продажи в каждои отдельном регионе.

ESRB рейтинг в нашем исследовании имеет следующие категории (названия взяты из интернета):
- T - «Подросткам»: Игра подходит для лиц от 13 лет.
- E - «Для всех»: Содержание вполне подходит для всех возрастов.
- M - «Для взрослых»: Материалы игры не подходят для лиц младше 17 лет.
- E10+ - «Для всех от 10 лет и старше»

In [ ]:
na_pivot_rating = data_pivot_func(na, 'rating', 'na_sales', 'count', 'sum')
na_pivot_rating

«Для взрослых» - самая популярная категория в продажах компьютерных игр в Северной Америке, доля продаж составляет 38.49%. «Для всех» не менее популярна с долей продаж в 18.71%. «Для всех от 10 лет и старше» на третьем месте с долей 12.59%. На последнем месте «Подросткам» - 11.28%.

In [ ]:
eu_pivot_rating = data_pivot_func(eu, 'rating', 'eu_sales', 'count', 'sum')
eu_pivot_rating

«Для взрослых» - самая популярная категория в продажах компьютерных игр в Европе, доля продаж составляет 38.03%. «Для всех» не менее популярна с долей продаж в 21.64%. «Для всех от 10 лет и старше» на третьем месте с долей 11.28%. На последнем месте «Подросткам» - 10.50%.

In [ ]:
jp_pivot_rating = data_pivot_func(jp, 'rating', 'jp_sales', 'count', 'sum')
jp_pivot_rating

В Японии большое количество игр без рейтинга имеют самую большую долю продаж в 58.68%. Следом за ней идут «Подросткам» (14.73%), «Для всех» (11.93%), «Для взрослых» (10.29%) и «Для всех от 10 лет и старше» (4.36%).

In [ ]:
other_pivot_rating = data_pivot_func(other, 'rating', 'other_sales', 'count', 'sum')
other_pivot_rating

«Для взрослых» - также самая популярная категория в продажах компьютерных игр в других странах, доля продаж составляет 42.32%. «Для всех» на втором месте с долей продаж 19.23%. «Подросткам» (10.82%) и «Для всех от 10 лет и старше»(10.14%) наименее популярны.

**Вывод:**

Нельзя сказать, как влияет определенный рейтинг на продажи, так как нужно уточнить, какие были продажи до присвоения рейтингов играм. 

Из данных выше можно сделать вывод, что кроме Японии, где наиболее популярные игры - без рейтинга, во всех регионах примерно на одном уровне лидируют игры с рейтингом «Для взрослых», наименее популярны: «Для всех», «Для всех от 10 лет и старше», «Подросткам».

### Проверка гипотез

#### Средние пользовательские рейтинги платформ Xbox и PC одинаковые

Ограничим датафрейм и оставим информацию только по необходимым столбцам для исследования и выведем столбчатую диаграмму по получившимся выборкам:

In [ ]:
data_hypothesis1 = data_new.loc[:, ['platform', 'user_score']]
data_xone = data_hypothesis1.query('user_score > -1 and platform == "XOne"')
data_pc = data_hypothesis1.query('user_score > -1 and platform == "PC"')

In [ ]:
ax = data_xone.plot(kind='hist',
               y='user_score', 
               bins=50, 
               grid=True,
               figsize=(10,5), 
               color="#32B5C9",
               rwidth=0.9,
               label='Платформа XOne',
)
data_pc.plot(kind='hist',
               y='user_score', 
               bins=50, 
               grid=True,
               figsize=(10,5), 
               color="#c94632",
               rwidth=0.9,
               label='Платформа PC',
               ax=ax,
);
plt.title('Пользовательские рейтинги платформ Xbox и PC', size=11)
plt.xlabel('Оценка пользователей (максимум 10)', labelpad=20, size=11)
plt.ylabel('Частота встречаемости', labelpad=20, size=11);

Проверим, равны ли размеры выборок:

In [ ]:
data_xone.shape[0]

In [ ]:
data_pc.shape[0]

Проверим, равны ли дисперсии:

In [ ]:
data_xone.var()['user_score']

In [ ]:
data_pc.var()['user_score']

- **Нулевая гипотеза** - средние пользовательские рейтинги платформ XOne и PC равны.
- **Альтернативная гипотеза** - средние пользовательские рейтинги платформ XOne и PC не равны.

Мы сравниваем две генеральные совокупности, поэтому сначала убедимся:
- Генеральные совокупности не зависят друг от друга: да, это так, потому что в первом датафрейме данные о пользовательских рейтингах по платформе XOne, во втором - по платформе PC.
- Выборочные средние нормально распределены.
- Дисперсии рассматриваемых генеральных совокупностей равны: не совсем так, что доказывают рассчеты ниже. И размеры выборок разные - уточним этот момент в проверке гипотез.

In [ ]:
print('среднее выборки по xone:', data_xone['user_score'].mean())
print('среднее выборки по pc:', data_pc['user_score'].mean())

alpha = 0.05

results = st.ttest_ind(data_xone['user_score'], data_pc['user_score'], \
                       alternative='two-sided', equal_var=False)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергать нулевую гипотезу')

Полученное значение p-value больше заданного уровня значимости, поэтому не отвергаем нулевую гипотезу. То есть, средние пользовательские рейтинги платформ Xbox и PC одинаковые.

#### Средние пользовательские рейтинги жанров Action и Sports разные

Ограничим датафрейм и оставим информацию только по необходимым столбцам для исследования и выведем столбчатую диаграмму по получившимся выборкам:

In [ ]:
data_hypothesis2 = data_new.loc[:, ['genre', 'user_score']]
data_action = data_hypothesis2.query('user_score > -1 and genre == "Action"')
data_sports = data_hypothesis2.query('user_score > -1 and genre == "Sports"')

In [ ]:
ax = data_action.plot(kind='hist',
               y='user_score', 
               bins=50, 
               grid=True,
               figsize=(10,5), 
               color="#32B5C9",
               rwidth=0.9,
               label='Жанр Action',
)
data_sports.plot(kind='hist',
               y='user_score', 
               bins=50, 
               grid=True,
               figsize=(10,5), 
               color="#c94632",
               rwidth=0.9,
               label='Жанр Sports',
               ax=ax,
);
plt.title('Пользовательские рейтинги жанров Action и Sports', size=11)
plt.xlabel('Оценка пользователей (максимум 10)', labelpad=20, size=11)
plt.ylabel('Частота встречаемости', labelpad=20, size=11);

Проверим, равны ли размеры выборок:

In [ ]:
data_action.shape[0]

In [ ]:
data_sports.shape[0]

Проверим, равны ли дисперсии:

In [ ]:
data_action.var()['user_score']

In [ ]:
data_sports.var()['user_score']

- **Нулевая гипотеза** - средние пользовательские рейтинги жанров Action и Sports равны.
- **Альтернативная гипотеза** - средние пользовательские рейтинги жанров Action и Sports не равны.

Мы сравниваем две генеральные совокупности, поэтому сначала убедимся:
- Генеральные совокупности не зависят друг от друга: да, это так, потому что в первом датафрейме данные о пользовательских рейтингах по жанру Action, во втором - по жанру Sports.
- Выборочные средние нормально распределены.
- Дисперсии рассматриваемых генеральных совокупностей равны: не совсем так, что доказывают рассчеты ниже. И размеры выборок разные - уточним этот момент в проверке гипотез.

In [ ]:
print('среднее выборки по жанру action:', data_action['user_score'].mean())
print('среднее выборки по жанру sports:', data_sports['user_score'].mean())

alpha = 0.05

results = st.ttest_ind(data_action['user_score'], data_sports['user_score'], \
                       alternative='two-sided', equal_var=False)

print(f'p-value: {results.pvalue}')

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Нет оснований отвергать нулевую гипотезу')

Полученное значение p-value меньше заданного уровня значимости, поэтому отвергаем нулевую гипотезу. То есть, средние пользовательские рейтинги жанров Action и Sports разные.

**Вывод:**

Были проверены 2 гипотезы на основе полученных датафреймов:

- средние пользовательские рейтинги платформ Xbox и PC одинаковые.
- средние пользовательские рейтинги жанров Action и Sports разные.

### Вывод

В данном исследовании были поставлены и выполнены следующие задачи:

- Изучена общая информация о датафреймах.
- Обработаны пропущенные значения.
- Обработаны неявные дубликаты.
- Изменен тип данных, где это необходимо.
- Добавлен в датафрейм новый столбец с суммарными продажами.
- Определен актуальный период для дальнейшего исследовательского анализа (2013-2016гг).
- Детально изучены параметры, влияющие на продажи компьютерных игр, с построением графиков и диаграмм рассеяния для каждого из них.
- Составлен портрет пользователя каждого региона: рассмотрены самые популярные платформы, жанры и установлено, влияет ли рейтинг ESRB на продажи в каждом отдельном регионе.
- Проверены гипотезы о равенстве средних пользовательских рейтингов различных платформ и жанров.

В результате исследовательского анализа данных можно сделать следующие итоговые выводы:

- Динамичное развитие игровой индустрии началось в 2000-х годах, в 2008-2009 гг приходится пик выпуска компьютерных игр - немного больше 1400 игр в каждый год.
- В среднем в течение 9 лет происходит "смена" платформ по их популярности.
- В период с 2013 по 2016 гг лидером по продажам была платформа PS4, XOne также показывает хорошие результаты по популярности, но прибыль различается от лидера в два раза. Общая тенденция следующая: к 2016 году все платформы устаревают и прибыль от продажи копий сильно снижается.
- Диаграмма рессеяния показала, что наибольшие продажи у X360, PS4, XOne, WiiU, Wii, наибольший разброс в продажах игр - на платформах X360, XOne, Wii. Выбросов достаточно много.
- Зависимость между отзывами критиков и продажами на платформах PS4, XOne, 3DS, WiiU, PC, PSV гораздо сильнее, чем корреляция между отзывами пользователей и продажами на платформах.
- Можно выделить жанры с высокими продажами: Shooter, Platform и Sports, и с низкими продажами (все остальные): Adventure, Strategy, Simulation, Role-Playing, Racing, Puzzle, Misc, Fighting, Action. Также можно обратить внимание на то, что есть некоторые жанры, игры по которым были выпущены в большом количестве, но при этом продажи не были высокими. Самым прибыльным считается жанр Shooter, что неудивительно, так как помимо покупки самой игры чаще всего игроки делают внутриигровые покупки на необходимую "экипировку" и т.д.

Составлен пользовательский портрет каждого региона:

Северная Америка:
- Самая популярная платформа по доле продаж компьютерных игр - PS4, 24.9%, следом за ней идет XOne, 21.65%, затем X360, 19.07%, PS3, 14.15%. На пятом месте 3DS с большим отрывом - 8.72%.
- Доля продаж компьтерных игр в жанре Action самая большая - 28.94%, на втором месте Shooter с разницей всего в 3% - 25.95%. Жанр Sports не мене популярен, хоть и доля продаж составляет 15.65%. Замыкают пятерку лидеров игры в жанрах Role-Playing(9.15%) и Misc (6.50%).
- «Для взрослых» - самая популярная категория в продажах компьютерных игр, доля продаж составляет 38.49%. «Для всех» не менее популярна с долей продаж в 18.71%. «Для всех от 10 лет и старше» на третьем месте с долей 12.59%. На последнем месте «Подросткам» - 11.28%.

Европа:

- Самая популярная платформа по доле продаж компьютерных игр также - PS4, но доля уже составила 36.47% - в Европе больше пользуется спросом платформа. Следом за ней идет PS3 с большим отрывом и долей продаж в 17.12%. На третьем месте XOne, 13.35%, затем X360, 10.82%. На пятом месте 3DS - 7.78%, она также непопулярна в этом регионе.
- Лидером по продажам также является игры в жанре Action (30.31%). Shooter (23.34%) на втором месте, Sports (15.88%) на третьем. Замыкают Role-Playing (8.16%) и Racing (5.22%).
- «Для взрослых» - самая популярная категория в продажах компьютерных игр, доля продаж составляет 38.03%. «Для всех» не менее популярна с долей продаж в 21.64%. «Для всех от 10 лет и старше» на третьем месте с долей 11.28%. На последнем месте «Подросткам» - 10.50%.

Япония:

- Самой популярной платформой становится 3DS - 51.77%. PS3 (15.56%), PSV (11.36%), PS4 (11.03%) вне конкуренции по сравнению с долей продаж 3DS, замыкает пятерку лидеров платформа WiiU с долей продаж 8.15%.
- Пользуются большим спросом игры в жанрах Role-Playing (35.56%) и Action (30.12%). С большим отрывом игры в жанрах Misc (7.16%), Fighting (5.35%) и Shooter (4.94%).
- Большое количество игр без рейтинга имеют самую большую долю продаж в 58.68%. Следом за ней идут «Подросткам» (14.73%), «Для всех» (11.93%), «Для взрослых» (10.29%) и «Для всех от 10 лет и старше» (4.36%).

Другие страны:

- Самая популярная платформа по доле продаж компьютерных игр сохраняется - PS4, с высоким процентом 43.29%, следом за ней идет PS3 (24.06%) и XOne (11.21%). X360 (9.86%) и 3DS (4.25%) замыкают пятерку лидеров.
- Популярными являются игры в жанрах Action (31.30%) и Shooter (26.28%). С небольшим отрывом на третьем месте Sports (17.20%). Замыкают пятерку лидеров игры в жанрах Role-Playing и Misc с долями продаж 7.44% и 4.93% соответственно.
- «Для взрослых» - также самая популярная категория в продажах компьютерных игр, доля продаж составляет 42.32%. «Для всех» на втором месте с долей продаж 19.23%. «Подросткам» (10.82%) и «Для всех от 10 лет и старше»(10.14%) наименее популярны.

Были проверены 2 гипотезы на основе полученных датафреймов:

- Средние пользовательские рейтинги платформ Xbox и PC одинаковые.
- Средние пользовательские рейтинги жанров Action и Sports разные.